In [ ]:
import configparser
import logging

import telegram
from flask import Flask, request
from telegram import ReplyKeyboardMarkup, InlineKeyboardMarkup, InlineKeyboardButton
from telegram.ext import Dispatcher, CommandHandler, MessageHandler, Filters, InlineQueryHandler
from time import sleep

class dict_to_obj(object):
    def __init__(self, d):
        for a, b in d.items():
            if isinstance(b, (list, tuple)):
                setattr(self, a, [dict_to_obj(x) if isinstance(x, dict) else x for x in b])
            else:
                setattr(self, a, dict_to_obj(b) if isinstance(b, dict) else b)


def obj_to_dict(obj):
    out = vars(obj)
    for key in out.keys():
        #print(type(out[key]))
        if type(out[key]) == type(data3):
            #print(vars(out[key]))
            out[key] = vars(out[key])
    return out



config = configparser.ConfigParser()
config.read('config.config')

# Enable logging
logging.basicConfig(format='%(asctime)s - %(name)s - %(levelname)s - %(message)s',
                    level=logging.INFO)
logger = logging.getLogger(__name__)

# Initial Flask app
app = Flask(__name__)

# Initial bot by Telegram access token
bot = telegram.Bot(token=(config['TELEGRAM']['ACCESS_TOKEN']))


#replace
welcome_message = """
歡迎使用本測試聊天機器人
本機器人作者為 EasonC13
將在上面佈署需要的功能
請使用 /help 來取得更多協助"""

testing_message = """
接下來將進行一系列的評分測試
幫助我們系統理解您的喜好
讓我們能推薦適合您的藝文活動喔！"""

hello_message = """
哈囉！
老客戶用 /start 指令
你是在哈囉嗎？
需要幫助歡迎選擇 /help
"""

help_message = """
R 
就是R
其實這個部分還沒寫啦
因為我們還沒做完嘛...

所以你還無法使用
但你還是可以用 /start 
來取得指引喔！"""

location_keyboard = telegram.KeyboardButton(text="傳送定位資訊", request_location=True)
phone_keyboard = telegram.KeyboardButton(text="📞綁定手機號碼", request_contact=True)
cancel_keyboard = telegram.KeyboardButton(text="取消")

KeyBoards = {
    "default" : ReplyKeyboardMarkup([['📡尋找在附近的藝文活動'],
                                     ['❓如何使用'],
                                     ['🛠個人設定'],
                                     ['✉️錯誤回報'], 
                                     ["❓關於本機器人"]]),
    "testing" : ReplyKeyboardMarkup([["好的，讓我們開始吧！"]]),
    "request_location" : ReplyKeyboardMarkup([[location_keyboard], [cancel_keyboard]]),
    "request_phone" : ReplyKeyboardMarkup([[phone_keyboard], [cancel_keyboard]]),
    "decide_go_or_not" : ReplyKeyboardMarkup([['我要去'],['沒興趣，請給我其他活動']]),
    "rate_score" : ReplyKeyboardMarkup([['1'], ['2'], ['3'], ['4'], ['5']]),
}
KeyBoards = dict_to_obj(KeyBoards)

def get_rate_inline_board():
    out = []
    for i in range(10, 50, 10):
        i = float(i/10)
        out.append(str(i))
    return [out]

rate_board = ReplyKeyboardMarkup(get_rate_inline_board(), resize_keyboard = True, one_time_keyboard = True, selective = True)


@app.route('/hook', methods=['POST'])
def webhook_handler():
    """Set route /hook with POST method will trigger this method."""
    if request.method == "POST":
        update = telegram.Update.de_json(request.get_json(force=True), bot)
        dispatcher.process_update(update)
    return 'ok'


def start_handler(bot, update):
    """Send a message when the command /start is issued."""
    already_in_use = False
    if already_in_use:
        update.message.reply_text(hello_message, reply_markup=KeyBoards.default)
    else:
        user_setup(update)
        
def user_setup(update):
    update.message.reply_text(welcome_message, reply_markup=KeyBoards.default)
    #update.message.reply_text(testing_message, reply_markup=KeyBoards.testing)
    #set User to testing mod
    #update.message.reply_text("請問你對 {這部影片} 的評分？", reply_markup=rate_board)

def help_handler(bot, update):
    """Send a message when the command /help is issued."""
    #update.message.reply_text(welcome_message, reply_markup=KeyBoards.default)
    update.message.reply_text(help_message, reply_markup = ReplyKeyboardMarkup([["...好喔"]]) )
    


def send_typing_action(update):
    """Show the bot is typing to user."""
    bot.send_chat_action(chat_id=update.message.chat.id, action=telegram.ChatAction.TYPING)
    
def location_handler(bot, update):
    message = None
    update.message.reply_text('請稍後，正在尋找您附近的藝文活動')
    send_typing_action(update)
    sleep(1)
    bot.send_location(update.message.chat.id, 25.028538, 121.5321779)
    update.message.reply_text('在青田街5巷6號，街頭藝人 Eason 正在表演虎鶴雙型', reply_markup = KeyBoards.decide_go_or_not)
    
    
def reply_processor(update):
    text = update.message.text
    print(text)
    reply_text = update.message.reply_text
    reply = None
    
    if "eason" in text:
        reply_text("叭叭！@IshengChen 有人叫你啊")
        
    elif text == "...好喔":
        reply_text("嘻嘻，謝謝你的包容", reply_markup=KeyBoards.default)
        
    elif text == "❓如何使用":
        reply_text("我...我才不告訴你呢...\n(其實是還沒寫啦...)", reply_markup=KeyBoards.default)
        
    elif text == "✉️錯誤回報":
        reply_text("請直接聯絡負責我的開發者 @IshengChen")
        
    elif text == "🛠個人設定":
        reply_text("目前尚未提供此功能\n(還沒寫...)", reply_markup=KeyBoards.default)
        
    elif text == "📡尋找在附近的藝文活動":
        reply_text("請問你想尋找的活動類型？(還沒寫)", reply_markup=KeyBoards.request_location)
        reply_text("請提供您的定位資訊：", reply_markup=KeyBoards.request_location)
        
    elif text == "取消":
        reply_text("收到", reply_markup=KeyBoards.default)
        
    elif text == "❓關於本機器人":
        reply_text("其實我應該要跟你講個感人的故事，\n關於本機器人如何幫助藝文產業起飛\n只是我們還沒寫QQ", reply_markup=KeyBoards.default)
    
    elif text == "沒興趣，請給我其他活動":
        sleep(1)
        bot.send_location(update.message.chat.id, 25.0301335, 121.5354529)
        reply_text("在大安森林公園露天音樂台，樂團 Eason 正在表演")
        
    elif text == "我要去":
        reply_text("祝您觀賞愉快，可直接點選上面的地圖做導航喔！", reply_markup=KeyBoards.default)
        sleep(10)
        reply_text("詢問結果", reply_markup=KeyBoards.rate_score)
        
    else:
        reply = "聽不懂QQ"
        
    return reply

def reply_handler(bot, update):
    """Reply message."""
    send_typing_action(update)
    reply = reply_processor(update)
    if reply != None:
        update.message.reply_text(reply, reply_markup=KeyBoards.default)


def error_handler(bot, update, error):
    """Log Errors caused by Updates."""
    logger.error('Update "%s" caused error "%s"', update, error)
    update.message.reply_text('對不起，系統錯誤\n(歡迎使用錯誤回報，告訴開發者，你剛剛輸入了什麼)')


# New a dispatcher for bot
dispatcher = Dispatcher(bot, None)

# Add handler for handling message, there are many kinds of message. For this handler, it particular handle text
# message.
dispatcher.add_handler(MessageHandler(Filters.text, reply_handler))
dispatcher.add_handler(CommandHandler('start', start_handler))
dispatcher.add_handler(CommandHandler('help', help_handler))
dispatcher.add_handler(MessageHandler(Filters.location, location_handler))
dispatcher.add_error_handler(error_handler)






if __name__ == "__main__":
    # Running server
    app.run(port = 12336)#, ssl_context=("/home/fio/myCA/server_crt.pem","/home/fio/myCA/server_key.pem"))
#要先去https://api.telegram.org/bot891550922:AAFl9Fb7oTgM-Ih_jT9JKfe__3ZsykitEE4/setWebhook?url=https://512c04e6.ngrok.io/hook


#https://api.telegram.org/bot891550922:AAFl9Fb7oTgM-Ih_jT9JKfe__3ZsykitEE4/setWebhook?url=https://ai.cto.tw:12336/hook
#把他Hook起來以後，才會有用

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


/home/fio/anaconda3/envs/chatbot/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3331: TelegramDeprecationWarning: Old Handler API is deprecated - see https://git.io/fxJuV for details
  exec(code_obj, self.user_global_ns, self.user_ns)
2020-02-09 11:25:18,329 - werkzeug - INFO -  * Running on http://127.0.0.1:12336/ (Press CTRL+C to quit)
2020-02-09 11:29:40,615 - werkzeug - INFO - 127.0.0.1 - - [09/Feb/2020 11:29:40] "POST /hook HTTP/1.1" 200 -


1


2020-02-09 11:29:45,035 - werkzeug - INFO - 127.0.0.1 - - [09/Feb/2020 11:29:45] "POST /hook HTTP/1.1" 200 -


Threrefore, welcome to ask me about the problem you face when programming.


...

2020-02-09 11:38:40,708 - werkzeug - INFO - 127.0.0.1 - - [09/Feb/2020 11:38:40] "POST /hook HTTP/1.1" 200 -
2020-02-09 11:38:41,113 - werkzeug - INFO - 127.0.0.1 - - [09/Feb/2020 11:38:41] "POST /hook HTTP/1.1" 200 -
2020-02-09 11:38:41,477 - telegram.vendor.ptb_urllib3.urllib3.connectionpool - WARNING - Connection pool is full, discarding connection: api.telegram.org


2


2020-02-09 11:38:42,098 - werkzeug - INFO - 127.0.0.1 - - [09/Feb/2020 11:38:42] "POST /hook HTTP/1.1" 200 -
2020-02-09 11:38:43,271 - telegram.vendor.ptb_urllib3.urllib3.connectionpool - WARNING - Connection pool is full, discarding connection: api.telegram.org


2


2020-02-09 11:38:43,816 - werkzeug - INFO - 127.0.0.1 - - [09/Feb/2020 11:38:43] "POST /hook HTTP/1.1" 200 -
